In [1]:
!pip install pycaret


In [2]:
import pandas as pd

# Upload CSV file
from google.colab import files
uploaded = files.upload()

# Load your dataset (adjust filename if necessary)
df = pd.read_csv('bank marketing campaign.csv')

# Preview the data
df.head()


Saving bank marketing campaign.csv to bank marketing campaign (1).csv


,Age,Job,Marital Status,Education,Has credit in default,Avg. credit balance,Has housing loan,Has personal loan,Contact type,Last contact day,Last contact month,Last contact duration (sec),Number of contacts,Days passed,Previous contacts,Outcome previous campaign,Subscribed deposit
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [4]:
from pycaret.classification import *

# Initialize PyCaret setup
clf1 = setup(
    data=df,
    target='Subscribed deposit',
    session_id=42,
    train_size=0.8,
    verbose=False
)


In [5]:
# Compare selected models and return the top 5
top_models = compare_models(include=['lr', 'rf', 'xgboost', 'lightgbm', 'svm'], n_select=5)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.9021,0.8880,0.9021,0.8872,0.8866,0.3808,0.4123,1.5630
lightgbm,Light Gradient Boosting Machine,0.9015,0.9075,0.9015,0.8906,0.8929,0.4374,0.4503,1.1990
rf,Random Forest Classifier,0.9002,0.9042,0.9002,0.8844,0.8784,0.3199,0.3718,0.6750
xgboost,Extreme Gradient Boosting,0.8993,0.9011,0.8993,0.8886,0.8915,0.4340,0.4440,0.3280
svm,SVM - Linear Kernel,0.8606,0.6043,0.8606,0.8403,0.8449,0.1814,0.1978,0.2450


Processing:   0%|          | 0/29 [00:00<?, ?it/s]

In [6]:
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score
import numpy as np

# Store results
eval_results = []

# Evaluate each top model
for model in top_models:
    name = model.__class__.__name__

    # Make predictions on test set
    preds = predict_model(model)

    y_true = preds['Subscribed deposit']
    y_pred = preds['prediction_label']

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=["yes", "no"])

    # Metrics
    acc = accuracy_score(y_true, y_pred)
    bal_acc = balanced_accuracy_score(y_true, y_pred)

    # Sensitivity (Recall) and Specificity for each class
    sens, spec = [], []
    for i in range(len(cm)):
        tp = cm[i, i]
        fn = cm[i, :].sum() - tp
        fp = cm[:, i].sum() - tp
        tn = cm.sum() - (tp + fn + fp)

        sensitivity = tp / (tp + fn) if (tp + fn) else 0
        specificity = tn / (tn + fp) if (tn + fp) else 0

        sens.append(sensitivity)
        spec.append(specificity)

    avg_sens = np.mean(sens)
    avg_spec = np.mean(spec)

    eval_results.append({
        'Model': name,
        'Accuracy': acc,
        'Balanced Accuracy': bal_acc,
        'Avg Sensitivity': avg_sens,
        'Avg Specificity': avg_spec
    })

# Convert to DataFrame and display
pd.DataFrame(eval_results)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.8939,0.8892,0.8939,0.8751,0.8787,0.3406,0.3627


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8950,0.9119,0.8950,0.8833,0.8875,0.4127,0.4195


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Random Forest Classifier,0.8928,0.8945,0.8928,0.8708,0.8725,0.2926,0.3267


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.8961,0.9061,0.8961,0.8857,0.8895,0.4271,0.4325


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,SVM - Linear Kernel,0.8464,0,0.8464,0.8392,0.8426,0.2085,0.2090


,Model,Accuracy,Balanced Accuracy,Avg Sensitivity,Avg Specificity
0,LogisticRegression,0.893923,0.634681,0.634681,0.634681
1,LGBMClassifier,0.895028,0.681324,0.681324,0.681324
2,RandomForestClassifier,0.892818,0.608956,0.608956,0.608956
3,XGBClassifier,0.896133,0.690315,0.690315,0.690315
4,SGDClassifier,0.846409,0.599473,0.599473,0.599473
